## Data Loading and Initial Preparation

In [46]:
import numpy as np
import pandas as pd
from datetime import datetime, date, time, timedelta

import warnings
warnings.filterwarnings('ignore')

start = datetime.now()

df = pd.read_csv('assignment_data.csv')

In [47]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1945726 entries, 0 to 1945725
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   provider_id  object
 1   detail       object
 2   source       object
 3   event_time   object
dtypes: object(4)
memory usage: 59.4+ MB


,provider_id,detail,source,event_time
0,6873f0af240706d65514ec247c64f330,{u'mme': False},job_responded,2017-09-01 00:00:00.000000
1,a5a021d997269c4f1846903ff0f5d737,"{u'lat': 18.9810108, u'long': 72.8187476}",location_tracking,2017-09-01 00:00:02.000000
2,77af6d6d6db530c389ec1014d557a01e,{u'mme': False},decline_request,2017-09-01 00:00:05.000000
3,7ea218c2b9a6dada27e0d5bdf5f896de,{u'mme': False},decline_request,2017-09-01 00:00:12.000000
4,0d7a9d9c00231885b413175980a9beca,"{u'lat': 12.9959477, u'long': 77.6527175}",location_tracking,2017-09-01 00:00:15.000000


### Splitting date and time

In [49]:
df_split = df['event_time'].str.split(expand=True)   
df_split.columns = ['Date', 'Time']
df_split['Time'] = df_split["Time"].str.replace(".000000","",regex=True)

# A dataframe with separate time and date columns
df_combined = pd.concat([df, df_split], axis=1)
df_combined

,provider_id,detail,source,event_time,Date,Time
0,6873f0af240706d65514ec247c64f330,{u'mme': False},job_responded,2017-09-01 00:00:00.000000,2017-09-01,00:00:00
1,a5a021d997269c4f1846903ff0f5d737,"{u'lat': 18.9810108, u'long': 72.8187476}",location_tracking,2017-09-01 00:00:02.000000,2017-09-01,00:00:02
2,77af6d6d6db530c389ec1014d557a01e,{u'mme': False},decline_request,2017-09-01 00:00:05.000000,2017-09-01,00:00:05
3,7ea218c2b9a6dada27e0d5bdf5f896de,{u'mme': False},decline_request,2017-09-01 00:00:12.000000,2017-09-01,00:00:12
4,0d7a9d9c00231885b413175980a9beca,"{u'lat': 12.9959477, u'long': 77.6527175}",location_tracking,2017-09-01 00:00:15.000000,2017-09-01,00:00:15
...,...,...,...,...,...,...
1945721,4fcaf6b3efd1057f5313cbeee45971ce,"{u'lat': 13.1017353, u'long': 77.5237836}",location_tracking,2017-09-30 23:59:24.000000,2017-09-30,23:59:24
1945722,071aeff24666fc1d2f0550df11446165,"{u'lat': 19.1423332, u'long': 72.8375924}",location_tracking,2017-09-30 23:59:25.000000,2017-09-30,23:59:25
1945723,93c11631d6844292b8eb64ae1335efe9,"{u'lat': 13.0037739, u'long': 77.6344158}",location_tracking,2017-09-30 23:59:29.000000,2017-09-30,23:59:29
1945724,80ce8c0b6f2c5c3ce45ef92211f86446,"{u'lat': 12.9319261, u'long': 77.5918504}",location_tracking,2017-09-30 23:59:33.000000,2017-09-30,23:59:33


## Time Adjustments Based on Rules

In [51]:
# Apply Rule 3: Events from midnight to 8 AM are considered at 8 AM.
df_processed = df_combined.copy()
df_processed.loc[df_processed["Time"].between('00:00:00', '08:00:00', inclusive='left'), "Time"] = '08:00:00'

In [52]:
# Designate online and offline status based on rules 4, 5, 7
# Rule 5: 'detail' is 'False' -> Offline
df_offline = df_processed[df_processed['detail'].str.contains('False', regex=False, na=False)].copy()
df_offline['Status'] = 'Offline'

# Rule 4: 'detail' is 'True' OR 'source' is 'Action on Job' -> Online
df_online = df_processed[(df_processed['detail'].str.contains('True', regex=False, na=False)) | 
                         (df_processed['source'].str.contains('Action on Job', regex=False, na=False))].copy()
df_online['Status'] = 'Online'

# Rule 7: If both online and offline then treat as online.
# This means if an event satisfies 'df_offline' and 'df_online' conditions,
# it should prioritize 'Online'.

# Create a temporary status priority: Online (1) > Offline (0)
df_temp = pd.concat([df_offline, df_online], ignore_index=True)
df_temp['status_priority'] = df_temp['Status'].map({'Online': 1, 'Offline': 0})

# Combine Date and Time into a single datetime object for precise sorting
df_temp['dt'] = pd.to_datetime(df_temp['Date'] + ' ' + df_temp['Time'])


### Generate the full daily time grid for all providers (8 AM to 7 PM hourly)

In [54]:
# Sort by provider, datetime, then by status priority to apply Rule 8 (Online preference for same timestamp)
df_temp = df_temp.sort_values(by=['provider_id', 'dt', 'status_priority'], ascending=[True, True, False])

# Drop duplicates, keeping the 'Online' status for same provider_id, dt (Rule 8)
tf2_events = df_temp.drop_duplicates(subset=['provider_id', 'dt'], keep='first').copy()
tf2_events.drop(columns=['detail', 'source', 'event_time', 'status_priority'], inplace=True)

all_providers = tf2_events['provider_id'].unique()
all_dates = pd.to_datetime(pd.date_range('2017-09-01', '2017-09-30')).date # All dates in Sept 2017

# Hourly points for the measurement period (8 AM, 9 AM, ..., 6 PM)
hourly_times = pd.to_datetime(pd.date_range('08:00:00', '18:00:00', freq='H').time.astype(str)).time


In [55]:
# Create a DataFrame for all possible (provider, date, hour_start) combinations
full_template = pd.MultiIndex.from_product([all_providers, all_dates, hourly_times],
                                           names=['provider_id', 'Date', 'Time']).to_frame(index=False)

full_template['dt'] = pd.to_datetime(full_template['Date'].astype(str) + ' ' + full_template['Time'].astype(str))
full_template['Status'] = np.nan # Status will be filled by forward-fill
full_template.info()
full_template.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5373390 entries, 0 to 5373389
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   provider_id  object        
 1   Date         object        
 2   Time         object        
 3   dt           datetime64[ns]
 4   Status       float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 205.0+ MB


,provider_id,Date,Time,dt,Status
0,00020da44b0a44e867de5f26c69886ac,2017-09-01,08:00:00,2017-09-01 08:00:00,NaN
1,00020da44b0a44e867de5f26c69886ac,2017-09-01,09:00:00,2017-09-01 09:00:00,NaN
2,00020da44b0a44e867de5f26c69886ac,2017-09-01,10:00:00,2017-09-01 10:00:00,NaN
3,00020da44b0a44e867de5f26c69886ac,2017-09-01,11:00:00,2017-09-01 11:00:00,NaN
4,00020da44b0a44e867de5f26c69886ac,2017-09-01,12:00:00,2017-09-01 12:00:00,NaN


In [56]:
# Combine actual events with the full template, and sort
# Sort by provider_id and the full datetime 'dt' to ensure correct forward-fill.
tf2 = pd.concat([tf2_events, full_template], ignore_index=True)
tf2 = tf2.sort_values(by=['provider_id', 'dt']).reset_index(drop=True)


### Forward-fill the Status

In [58]:
# Forward-fill the 'Status' for all time points within each provider group.
# This propagates the last known status to all subsequent time points until a new event.
tf2['Status'] = tf2.groupby('provider_id')['Status'].ffill()


In [59]:
# --- Correction for initial 8 AM status and final calculation of Online_time ---

tf2.loc[(tf2['dt'].dt.hour == 8) & (tf2['dt'].dt.minute == 0) & (tf2['dt'].dt.second == 0) & (tf2['Status'].isna()), 'Status'] = 'Offline'
# Re-ffill after setting these specific 8 AM initial statuses, to propagate any new 'Offline' states.
tf2['Status'] = tf2.groupby('provider_id')['Status'].ffill()


In [60]:
# Calculate next event time and status *within each provider_id group*.
tf2['next_dt'] = tf2.groupby('provider_id')['dt'].shift(-1)
tf2['next_status'] = tf2.groupby('provider_id')['Status'].shift(-1)

In [108]:
# Define the start and end boundaries for the active measurement period for each day.
tf2['day_start_8am'] = pd.to_datetime(tf2['Date'].astype(str) + ' 08:00:00')
tf2['day_end_7pm'] = pd.to_datetime(tf2['Date'].astype(str) + ' 19:00:00')


# Initialize 'Online_time' column
tf2['Online_time'] = 0

In [109]:
tf2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6042635 entries, 0 to 6042634
Data columns (total 13 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   provider_id            object        
 1   Date                   object        
 2   Time                   object        
 3   Status                 object        
 4   dt                     datetime64[ns]
 5   next_dt                datetime64[ns]
 6   next_status            object        
 7   day_start_8am          datetime64[ns]
 8   day_end_7pm            datetime64[ns]
 9   Online_time            int64         
 10  segment_start_clipped  datetime64[ns]
 11  segment_end_clipped    datetime64[ns]
 12  Hour_start             datetime64[ns]
dtypes: datetime64[ns](7), int64(1), object(5)
memory usage: 599.3+ MB


## Online/Offline Classification

- Calculate 'Online_time' for each segment based on rules:
    1. Status is 'Online' for the current segment.
    2. Segment starts effectively at or after 8 AM.
    3. Segment ends effectively at or before 7 PM.
    4. Clip the segment's start and end times to these boundaries.

In [114]:
# Determine the effective start of the online segment (max of event time or 8 AM)
tf2['segment_start_clipped'] = tf2[['dt', 'day_start_8am']].max(axis=1)

# Determine the effective end of the online segment (min of next event time or 7 PM)
tf2['segment_end_clipped'] = tf2[['next_dt', 'day_end_7pm']].min(axis=1)

# Calculate duration for segments where the current status is 'Online'
# (segment_end_clipped - segment_start_clipped) should be positive
tf2.loc[tf2['Status'] == 'Online', 'Online_time'] = \
    (tf2['segment_end_clipped'] - tf2['segment_start_clipped']).dt.total_seconds().clip(lower=0)

# Exclude Online time if segment starts exactly at 8:00:00
# If a segment is Online AND its effective start time is exactly 8:00:00, set its Online_time to 0.
mask_online_at_8am_exact_start = (tf2['Status'] == 'Online') & \
                                 (tf2['segment_start_clipped'].dt.time == time(8, 0, 0))

tf2.loc[mask_online_at_8am_exact_start, 'Online_time'] = 0

In [116]:
# Ensure no online time is counted if the segment itself starts at or after 7 PM
tf2.loc[tf2['segment_start_clipped'].dt.hour >= 19, 'Online_time'] = 0

# Also, if the status is explicitly 'Offline', the online time for that segment is 0.
# (This is mostly redundant with the .loc above but good for clarity).
tf2.loc[tf2['Status'] == 'Offline', 'Online_time'] = 0
tf2['Hour_start'] = tf2['dt'].dt.floor('H') #so that no issues when grouping in 'res'

In [118]:
print("\n--- DEBUGGING: 0037dfffff8b03bbdf366a263735e84b on 2017-09-04 (Intermediate tf2 state) ---")
debug_provider_id = '0037dfffff8b03bbdf366a263735e84b'
debug_date = '2017-09-04'
debug_df_tf2 = tf2.loc[(tf2['provider_id'] == debug_provider_id) &
                       (tf2['Date'].astype(str) == debug_date)].copy()
# Display relevant columns for debugging
print(debug_df_tf2[['dt','Hour_start', 'Status', 'next_dt', 'next_status', 'day_start_8am', 'day_end_7pm', 'segment_start_clipped', 'segment_end_clipped', 'Online_time']].to_string())


--- DEBUGGING: 0037dfffff8b03bbdf366a263735e84b on 2017-09-04 (Intermediate tf2 state) ---
                      dt          Hour_start   Status             next_dt next_status       day_start_8am         day_end_7pm segment_start_clipped segment_end_clipped  Online_time
6725 2017-09-04 08:00:00 2017-09-04 08:00:00   Online 2017-09-04 08:00:19      Online 2017-09-04 08:00:00 2017-09-04 19:00:00   2017-09-04 08:00:00 2017-09-04 08:00:19            0
6726 2017-09-04 08:00:19 2017-09-04 08:00:00   Online 2017-09-04 08:29:39     Offline 2017-09-04 08:00:00 2017-09-04 19:00:00   2017-09-04 08:00:19 2017-09-04 08:29:39         1760
6727 2017-09-04 08:29:39 2017-09-04 08:00:00  Offline 2017-09-04 08:44:36     Offline 2017-09-04 08:00:00 2017-09-04 19:00:00   2017-09-04 08:29:39 2017-09-04 08:44:36            0
6728 2017-09-04 08:44:36 2017-09-04 08:00:00  Offline 2017-09-04 09:00:00     Offline 2017-09-04 08:00:00 2017-09-04 19:00:00   2017-09-04 08:44:36 2017-09-04 09:00:00            0
672

In [120]:
tf2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6042635 entries, 0 to 6042634
Data columns (total 13 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   provider_id            object        
 1   Date                   object        
 2   Time                   object        
 3   Status                 object        
 4   dt                     datetime64[ns]
 5   next_dt                datetime64[ns]
 6   next_status            object        
 7   day_start_8am          datetime64[ns]
 8   day_end_7pm            datetime64[ns]
 9   Online_time            int64         
 10  segment_start_clipped  datetime64[ns]
 11  segment_end_clipped    datetime64[ns]
 12  Hour_start             datetime64[ns]
dtypes: datetime64[ns](7), int64(1), object(5)
memory usage: 599.3+ MB


### Aggregation and Final Output Formatting

In [123]:
# Group by provider, date, and hourly bins based on the 'dt' column
# Sum the 'Online_time' for each hourly bin.
res = tf2.groupby(['provider_id', pd.Grouper(key='dt', freq='H')])['Online_time'].sum().reset_index()

In [125]:
# --- DEBUGGING: Check 'res' DataFrame before final merge ---
print(res.loc[(res['provider_id'] == debug_provider_id) & (res['dt'].dt.date.astype(str) == debug_date)].to_string())

                           provider_id                  dt  Online_time
6376  0037dfffff8b03bbdf366a263735e84b 2017-09-04 08:00:00         1760
6377  0037dfffff8b03bbdf366a263735e84b 2017-09-04 09:00:00         3228
6378  0037dfffff8b03bbdf366a263735e84b 2017-09-04 10:00:00         3600
6379  0037dfffff8b03bbdf366a263735e84b 2017-09-04 11:00:00         3600
6380  0037dfffff8b03bbdf366a263735e84b 2017-09-04 12:00:00         1414
6381  0037dfffff8b03bbdf366a263735e84b 2017-09-04 13:00:00            0
6382  0037dfffff8b03bbdf366a263735e84b 2017-09-04 14:00:00            0
6383  0037dfffff8b03bbdf366a263735e84b 2017-09-04 15:00:00            0
6384  0037dfffff8b03bbdf366a263735e84b 2017-09-04 16:00:00            0
6385  0037dfffff8b03bbdf366a263735e84b 2017-09-04 17:00:00         1289
6386  0037dfffff8b03bbdf366a263735e84b 2017-09-04 18:00:00         3600


In [127]:
res['Date'] = res['dt'].dt.date

In [128]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5447036 entries, 0 to 5447035
Data columns (total 4 columns):
 #   Column       Dtype         
---  ------       -----         
 0   provider_id  object        
 1   dt           datetime64[ns]
 2   Online_time  int64         
 3   Date         object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 166.2+ MB


In [131]:
# Rename the grouped 'dt' column to 'Hour_start'
res.rename(columns={'dt': 'Hour_start'}, inplace=True)

# Calculate 'Hour_end'
res['Hour_end'] = res['Hour_start'] + pd.Timedelta(hours=1)

# Extract just the hour number for 'Hour_start' and 'Hour_end'
res['Hour_start'] = res['Hour_start'].dt.hour
res['Hour_end'] = res['Hour_end'].dt.hour

# Cast 'Online_time' to integer as it represents seconds.
res['Seconds online'] = res['Online_time'].astype(int)
res.drop(columns=['Online_time'], inplace=True) # Drop the original float column

In [132]:
# The 'full_template' uses datetime.time objects for the 'Time' column, so we'll adjust for merging
full_template['Hour_start'] = full_template['Time'].apply(lambda x: x.hour)
final_output = pd.merge(full_template[['provider_id', 'Date', 'Hour_start']], res,
                       on=['provider_id', 'Date', 'Hour_start'], how='left')

# Fill NaN 'Seconds online' with 0 for hours where no online activity was recorded.
final_output['Seconds online'] = final_output['Seconds online'].fillna(0).astype(int)

# Add 'Hour_end' to the final output
final_output['Hour_end'] = final_output['Hour_start'] + 1

# Ensure 'Hour_end' does not exceed 19 (7 PM)
final_output.loc[final_output['Hour_start'] == 18, 'Hour_end'] = 19


## Final Sorting, Cleaning and Result

In [136]:
final_output = final_output.sort_values(by=['provider_id', 'Date', 'Hour_start']).reset_index(drop=True)

end = datetime.now()
print(f"Total processing time: {end - start}")

# Display results for a specific provider and date for verification
print("\nSample results for 0037dfffff8b03bbdf366a263735e84b on 2017-09-04:")
print(final_output.loc[(final_output['provider_id'] == '0037dfffff8b03bbdf366a263735e84b') & 
                       (final_output['Date'].astype(str) == '2017-09-04')])

print(f"\nFinal output shape: {final_output.shape}")

Total processing time: 0:08:56.335607

Sample results for 0037dfffff8b03bbdf366a263735e84b on 2017-09-04:
                           provider_id        Date  Hour_start  Hour_end  \
6303  0037dfffff8b03bbdf366a263735e84b  2017-09-04           8         9   
6304  0037dfffff8b03bbdf366a263735e84b  2017-09-04           9        10   
6305  0037dfffff8b03bbdf366a263735e84b  2017-09-04          10        11   
6306  0037dfffff8b03bbdf366a263735e84b  2017-09-04          11        12   
6307  0037dfffff8b03bbdf366a263735e84b  2017-09-04          12        13   
6308  0037dfffff8b03bbdf366a263735e84b  2017-09-04          13        14   
6309  0037dfffff8b03bbdf366a263735e84b  2017-09-04          14        15   
6310  0037dfffff8b03bbdf366a263735e84b  2017-09-04          15        16   
6311  0037dfffff8b03bbdf366a263735e84b  2017-09-04          16        17   
6312  0037dfffff8b03bbdf366a263735e84b  2017-09-04          17        18   
6313  0037dfffff8b03bbdf366a263735e84b  2017-09-04        

### Assigned Metrics added

In [139]:
# 1. No. of online events (from cleaned initial events)
num_online_events = tf2_events[tf2_events['Status'] == 'Online'].shape[0]
print(f"1. No. of online events: {num_online_events}")

# 2. No. of offline events (from cleaned initial events)
num_offline_events = tf2_events[tf2_events['Status'] == 'Offline'].shape[0]
print(f"2. No. of offline events: {num_offline_events}")

# 3. No. of providers left after removing a) error rows b) events on or after 7 PM
# These are the unique providers that were processed and are present in the final output.
num_providers_processed = final_output['provider_id'].nunique()
print(f"3. No. of providers processed: {num_providers_processed}") # Renamed from "left after removing errors/7 PM" for clarity

# 4. No. of rows in final output data frame (No. of providers * 30 * 11)
expected_rows = len(all_providers) * 30 * 11 # 11 hours are 8-9, ..., 18-19 (7 PM reset)
actual_rows_final_output = final_output.shape[0]
print(f"4. No. of rows in final output data frame: {actual_rows_final_output} (Expected: {expected_rows})")

# 5. No. of rows in final output where seconds online > 0
rows_with_online_activity = final_output[final_output['Seconds online'] > 0].shape[0]
print(f"5. No. of rows in final output where seconds online > 0: {rows_with_online_activity}")

1. No. of online events: 126734
2. No. of offline events: 542511
3. No. of providers processed: 16283
4. No. of rows in final output data frame: 5373390 (Expected: 5373390)
5. No. of rows in final output where seconds online > 0: 233353


In [141]:
# Save the final result to CSV
final_output.to_csv('provider_data_detail.csv', index=False)